In [0]:
EXTERNAL_LOC_SAVE_PATH = "s3://google-fit-data-raw-12e34974-eb44-4fb4-ad47-49a2b7455430/activity-metric-plots/"
CLEAN_UP_LOCATION = "/Volumes/google_fit/gold/activity_metrics_plots/"

In [0]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import os

In [0]:
def clean_up_images():
    global CLEAN_UP_LOCATION
    for file_info in dbutils.fs.ls(CLEAN_UP_LOCATION):
        if file_info.name.endswith(".png"):
            dbutils.fs.rm(file_info.path)
            # print(file_info.path)
    print(f"All images removed from the path: {CLEAN_UP_LOCATION}")
        

In [0]:
def remove_existing_files_in_s3():
    global EXTERNAL_LOC_SAVE_PATH
    for file_info in dbutils.fs.ls(EXTERNAL_LOC_SAVE_PATH):
        dbutils.fs.rm(EXTERNAL_LOC_SAVE_PATH, recurse= True)
    print(f"All plots files removed from S3 bucket: {EXTERNAL_LOC_SAVE_PATH}")
        

In [0]:
def move_files_to_s3():
    global CLEAN_UP_LOCATION
    global EXTERNAL_LOC_SAVE_PATH
    for file_info in dbutils.fs.ls(CLEAN_UP_LOCATION):
        if not file_info.name.endswith(".png"):
            dbutils.fs.mv(file_info.path, os.path.join(EXTERNAL_LOC_SAVE_PATH, file_info.name))
            # print(file_info.path, os.path.join(EXTERNAL_LOC_SAVE_PATH, file_info.name))
    print(f"All files moved to S3 bucket: {EXTERNAL_LOC_SAVE_PATH}")
        

In [0]:
remove_existing_files_in_s3()

In [0]:
with ThreadPoolExecutor() as e:
    futures = [e.submit(clean_up_images), e.submit(move_files_to_s3)]

for future in as_completed(futures):
    future.result()
